# Pipelining Queries

## What does pipelining mean?

You have learned that you can use the WITH clause to redefine the scope for the query:

```cypher
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m  LIMIT 5
// possibly do more with the five m nodes
RETURN m.title AS movies
```

You can do something with the five nodes by adding another `MATCH` clause:

```cypher
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m  LIMIT 5
MATCH (d:Person)-[:DIRECTED]->(m)
RETURN d.name AS director,
m.title AS movies
```

## Using WITH for aggregation

A very common use of `WITH` is to aggregate data so that intermediate results can be used to create the final returned data or to pass the intermediate results to the next part of the query.

```cypher
MATCH (:Movie {title: 'Toy Story'})-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(m)
WHERE m.imdbRating IS NOT NULL
WITH g.name AS genre,
count(m) AS moviesInCommon,
sum(m.imdbRating) AS total
RETURN genre, moviesInCommon,
total/moviesInCommon AS score
ORDER By score DESC
```

Here is another example that shows aggregation and pipelining:

```cypher
MATCH (u:User {name: "Misty Williams"})-[r:RATED]->(:Movie)
WITH u, avg(r.rating) AS average
MATCH (u)-[r:RATED]->(m:Movie)
WHERE r.rating > average
RETURN average , m.title AS movie,
r.rating as rating
ORDER BY rating DESC
```

## Using WITH for collecting

Another common use for the `WITH` clause is to collect results into a list that will be returned:

```cypher
MATCH (m:Movie)--(a:Actor)
WHERE m.title CONTAINS 'New York'
WITH m, collect (a.name) AS actors,
count(*) AS numActors
RETURN m.title AS movieTitle, actors
ORDER BY numActors DESC
```

Here is another example where we perform a 2-step aggregation for collecting a list of maps:

```cypher
MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor)
WHERE m.title CONTAINS 'New York'
WITH m, collect (a.name) AS actors,
count(*) AS numActors
ORDER BY numActors DESC
RETURN collect(m { .title, actors, numActors }) AS movies
```

## Using LIMIT early

A best practice is to execute queries that minimize the number of rows processed in the query. One way to do that is to limit early in the query. This also helps in reducing the number of properties loaded from the database too early.

```cypher
PROFILE MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.born.year = 1980
WITH p,
collect(m.title) AS movies LIMIT 3
RETURN p.name AS actor, movies
```

A better way to do this query is to limit early. This query will perform slightly better because LIMIT is moved up in the query:

```cypher
PROFILE MATCH (p:Actor)
WHERE p.born.year = 1980
WITH p  LIMIT 3
MATCH (p)-[:ACTED_IN]->(m:Movie)
WITH p, collect(m.title) AS movies
RETURN p.name AS actor,  movies
```

## Use DISTINCT when necessary

Here is an example of a query that retrieves three actors, then collects the names of the genres for the movies that actor acted in.

```cypher
MATCH (p:Actor)
WHERE p.born.year = 1980
WITH p  LIMIT 3
MATCH (p)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WITH p, collect(g.name) AS genres
RETURN p.name AS actor, genres
```

Notice that for this query, the collected genre names are repeated. You would want to ensure that they are not duplicated as follows:

```cypher
MATCH (p:Actor)
WHERE p.born.year = 1980
WITH p  LIMIT 3
MATCH (p)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WITH p, collect(DISTINCT g.name) AS genres
RETURN p.name AS actor, genres
```